<a href="https://colab.research.google.com/github/gandalf1819/SF-Opioid-Crisis/blob/master/SF_drug_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys,os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
sns.set()

In [3]:
d_crime = pd.read_csv("/content/gdrive/My Drive/SF dataset/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")
d_crime.columns

Index(['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
       'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId'],
      dtype='object')

In [0]:
np.random.seed(100)
random_d_crime=d_crime.sample(2215024)
train_size=int(0.67 *2215024)
train_df=random_d_crime[:train_size]
test_df=random_d_crime[train_size:]

In [5]:
train_df.columns

Index(['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
       'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId'],
      dtype='object')

In [6]:
test_df.columns

Index(['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
       'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId'],
      dtype='object')

In [7]:
test_df['Id'] = test_df.index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
train_df=train_df.drop(['IncidntNum', 'PdId', 'Location'], axis=1)

In [9]:
train_df.sample(5)

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y
1987620,SUSPICIOUS OCC,INVESTIGATIVE DETENTION,Friday,01/01/2016,04:20,CENTRAL,"ARREST, BOOKED",200 Block of POST ST,-122.406022,37.788649
892776,OTHER OFFENSES,RESISTING ARREST,Sunday,06/13/2004,11:30,CENTRAL,"ARREST, BOOKED",BROADWAY ST / GRANT AV,-122.407018,37.797823
187188,BURGLARY,"BURGLARY, FORCIBLE ENTRY",Saturday,05/09/2015,10:22,CENTRAL,NONE,200 Block of SUTTER ST,-122.404828,37.789760
24997,OTHER OFFENSES,RESISTING ARREST,Thursday,03/06/2014,09:39,BAYVIEW,"ARREST, BOOKED",REVERE AV / GRIFFITH ST,-122.380820,37.726365
669340,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,08/17/2007,14:45,SOUTHERN,NONE,0 Block of BEALE ST,-122.396648,37.791740


In [0]:
test_df=test_df.drop(['IncidntNum', 'PdId', 'Location'], axis=1)

In [11]:
test_df.sample(5)

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Id
94105,OTHER OFFENSES,TRAFFIC VIOLATION,Sunday,05/18/2014,13:48,MISSION,"ARREST, CITED",1000 Block of POTRERO AV,-122.406681,37.755343,94105
1512870,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Tuesday,05/03/2005,14:20,PARK,"ARREST, BOOKED",ASHBURY ST / FELL ST,-122.447498,37.772791,1512870
497923,OTHER OFFENSES,PROBATION VIOLATION,Monday,02/16/2004,12:52,BAYVIEW,"ARREST, BOOKED",1500 Block of MC KINNON AV,-122.388278,37.735395,497923
188903,BURGLARY,"BURGLARY,STORE UNDER CONSTRUCTION, FORCIBLE ENTRY",Friday,05/15/2015,23:00,INGLESIDE,NONE,1600 Block of CHURCH ST,-122.426639,37.744103,188903
1451444,BURGLARY,"BURGLARY OF APARTMENT HOUSE, UNLAWFUL ENTRY",Friday,02/06/2009,14:17,PARK,"ARREST, BOOKED",1600 Block of GOLDEN GATE AV,-122.437307,37.778939,1451444


In [12]:
train_df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y'],
      dtype='object')

**Convert 'Date' and 'Time' columns to 'DateTime' columns to identify crimes across different times of the day**

In [0]:
train_df['DateTime'] = train_df.Date + " " + train_df.Time
train_df['DateTime'] = pd.to_datetime(train_df['DateTime'], format='%m/%d/%Y %H:%M')

test_df['DateTime'] = test_df.Date + " " + test_df.Time
test_df['DateTime'] = pd.to_datetime(test_df['DateTime'], format='%m/%d/%Y %H:%M')

In [0]:
train_df = train_df.drop(['Date', 'Time'], axis=1)
test_df = test_df.drop(['Date', 'Time'], axis=1)

Develop a feature **'IsDay'** to identify whether the crime was committed during day or night

In [15]:
train_df['IsDay'] = 0
train_df.loc[ (train_df.DateTime.dt.hour > 6) & (train_df.DateTime.dt.hour < 20), 'IsDay' ] = 1
test_df['IsDay'] = 0
test_df.loc[ (test_df.DateTime.dt.hour > 6) & (test_df.DateTime.dt.hour < 20), 'IsDay' ] = 1

train_df.sample(3)

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,DateTime,IsDay
1436649,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,NORTHERN,"ARREST, BOOKED",1400 Block of POLK ST,-122.420661,37.790100,2006-08-09 17:14:00,1
1508667,WARRANTS,WARRANT ARREST,Saturday,TARAVAL,"ARREST, BOOKED",JUDAH ST / LA PLAYA ST,-122.509139,37.760279,2004-12-04 10:05:00,1
1976663,ASSAULT,BATTERY WITH SERIOUS INJURIES,Friday,SOUTHERN,NONE,5TH ST / HOWARD ST,-122.404934,37.781499,2017-01-20 02:00:00,0



Create **'Month'**, **'Year'** and **'DayOfWeekInt'** columns


Map 'DayOfWeek' to Integer

In [16]:
days_to_int_dic = {
        'Monday': 1,
        'Tuesday': 2,
        'Wednesday': 3,
        'Thursday': 4,
        'Friday': 5,
        'Saturday': 6,
        'Sunday': 7,
}
train_df['DayOfWeek'] = train_df['DayOfWeek'].map(days_to_int_dic)
test_df ['DayOfWeek'] = test_df ['DayOfWeek'].map(days_to_int_dic)

train_df.DayOfWeek.unique()

array([5, 6, 7, 3, 1, 2, 4])

Create Hour, Month and Year Columns

In [17]:
train_df['Hour'] = train_df.DateTime.dt.hour
train_df['Month'] = train_df.DateTime.dt.month
train_df['Year'] = train_df.DateTime.dt.year
train_df['Year'] = train_df['Year'] - 2000 # The Algorithm doesn't know the difference

test_df['Hour'] = test_df.DateTime.dt.hour
test_df['Month'] = test_df.DateTime.dt.month
test_df['Year'] = test_df.DateTime.dt.year
test_df['Year'] = test_df['Year'] - 2000 # The Algorithm doesn't know the difference

train_df.sample(1)

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,DateTime,IsDay,Hour,Month,Year
714500,OTHER OFFENSES,PAROLE VIOLATION,5,PARK,"ARREST, BOOKED",100 Block of JOHNFKENNEDY DR,-122.457508,37.771133,2012-06-29 17:53:00,1,17,6,12



Deal with the cyclic characteristic of Months and Days of Week

In [18]:
train_df['HourCos'] = np.cos((train_df['Hour']*2*np.pi)/24 )
train_df['DayOfWeekCos'] = np.cos((train_df['DayOfWeek']*2*np.pi)/7 )
train_df['MonthCos'] = np.cos((train_df['Month']*2*np.pi)/12 )

test_df['HourCos'] = np.cos((test_df['Hour']*2*np.pi)/24 )
test_df['DayOfWeekCos'] = np.cos((test_df['DayOfWeek']*2*np.pi)/7 )
test_df['MonthCos'] = np.cos((test_df['Month']*2*np.pi)/12 )

train_df.sample(1)

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,DateTime,IsDay,Hour,Month,Year,HourCos,DayOfWeekCos,MonthCos
751684,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,5,TENDERLOIN,"ARREST, BOOKED",400 Block of JONES ST,-122.412857,37.78558,2003-10-03 23:29:00,0,23,10,3,0.965926,-0.222521,0.5



Dummy Encoding of 'PdDistrict'

In [19]:
train_df = pd.get_dummies(train_df, columns=['PdDistrict'])
test_df  = pd.get_dummies(test_df,  columns=['PdDistrict'])
train_df.sample(2)

,Category,Descript,DayOfWeek,Resolution,Address,X,Y,DateTime,IsDay,Hour,Month,Year,HourCos,DayOfWeekCos,MonthCos,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
1766593,FORGERY/COUNTERFEITING,"CHECKS, MAKE OR PASS FICTITIOUS",5,NONE,SILVER AV / REVERE AV,-122.398556,37.736434,2006-05-05 06:40:00,0,6,5,6,6.123234e-17,-0.222521,-0.866025,1,0,0,0,0,0,0,0,0,0
1796478,SUSPICIOUS OCC,SUSPICIOUS PERSON,3,NONE,0 Block of SOUTHPARK AV,-122.393093,37.782336,2006-01-04 18:10:00,1,18,1,6,-1.836970e-16,-0.900969,0.866025,0,0,0,0,0,0,0,1,0,0


Label Encoding of **'Category'**

In [20]:
from sklearn.preprocessing import LabelEncoder

cat_le = LabelEncoder()
train_df['CategoryInt'] = pd.Series(cat_le.fit_transform(train_df.Category))
train_df.sample(5)

,Category,Descript,DayOfWeek,Resolution,Address,X,Y,DateTime,IsDay,Hour,Month,Year,HourCos,DayOfWeekCos,MonthCos,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,CategoryInt
1638162,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",6,PSYCHOPATHIC CASE,800 Block of DUBOCE AV,-122.437112,37.769018,2012-09-29 01:10:00,0,1,9,12,0.965926,0.623490,-1.836970e-16,0,0,0,0,0,1,0,0,0,0,NaN
1911474,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",7,NONE,3600 Block of SACRAMENTO ST,-122.452774,37.787392,2017-09-10 20:00:00,0,20,9,17,0.500000,1.000000,-1.836970e-16,0,0,0,0,0,0,1,0,0,0,NaN
256130,DRUG/NARCOTIC,POSSESSION OF BASE/ROCK COCAINE,2,NONE,OFARRELL ST / HYDE ST,-122.416262,37.785375,2010-03-30 22:50:00,0,22,3,10,0.866025,-0.222521,6.123234e-17,0,0,0,0,0,0,0,0,0,1,16.0
1491366,BURGLARY,"BURGLARY,STORE UNDER CONSTRUCTION, UNLAWFUL ENTRY",3,NONE,500 Block of SUTTER ST,-122.409424,37.789065,2004-12-08 01:45:00,0,1,12,4,0.965926,-0.900969,1.000000e+00,0,1,0,0,0,0,0,0,0,0,NaN
2010495,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,7,NONE,700 Block of GREAT HY,-122.511079,37.773992,2016-04-24 14:30:00,1,14,4,16,-0.866025,1.000000,-5.000000e-01,0,0,0,0,0,0,1,0,0,0,NaN


In [0]:
train_df['InIntersection'] = 1
train_df.loc[train_df.Address.str.contains('Block'), 'InIntersection'] = 0

test_df['InIntersection'] = 1
test_df.loc[test_df.Address.str.contains('Block'), 'InIntersection'] = 0

In [22]:
train_df.sample(10)

,Category,Descript,DayOfWeek,Resolution,Address,X,Y,DateTime,IsDay,Hour,Month,Year,HourCos,DayOfWeekCos,MonthCos,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,CategoryInt,InIntersection
122704,DRUG/NARCOTIC,POSSESSION OF CONTROLLED SUBSTANCE,2,"ARREST, BOOKED",200 Block of GOLDEN GATE AV,-122.415121,37.781759,2014-09-09 14:04:00,1,14,9,14,-8.660254e-01,-0.222521,-1.836970e-16,0,0,0,0,0,0,0,0,0,1,21.0,0
1157645,VEHICLE THEFT,"VEHICLE, RECOVERED, AUTO",6,NONE,3300 Block of OCTAVIA ST,-122.430020,37.803051,2003-08-16 08:00:00,1,8,8,3,-5.000000e-01,0.623490,-5.000000e-01,0,0,0,0,1,0,0,0,0,0,16.0,0
726120,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,6,"ARREST, BOOKED",300 Block of EDDY ST,-122.412931,37.783834,2010-09-11 23:25:00,0,23,9,10,9.659258e-01,0.623490,-1.836970e-16,0,0,0,0,0,0,0,0,0,1,37.0,0
2118978,OTHER OFFENSES,TRAFFIC VIOLATION,1,"ARREST, BOOKED",24TH ST / ALABAMA ST,-122.410963,37.752685,2016-05-30 18:51:00,1,18,5,16,-1.836970e-16,0.623490,-8.660254e-01,0,0,0,1,0,0,0,0,0,0,NaN,1
1214220,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",5,"ARREST, CITED",KEITH ST / REVERE AV,-122.388295,37.730608,2008-03-07 16:15:00,1,16,3,8,-5.000000e-01,-0.222521,6.123234e-17,1,0,0,0,0,0,0,0,0,0,7.0,1
1479990,VEHICLE THEFT,STOLEN AUTOMOBILE,7,NONE,28TH ST / SANCHEZ ST,-122.429038,37.745166,2012-07-29 20:00:00,0,20,7,12,5.000000e-01,1.000000,-8.660254e-01,0,0,1,0,0,0,0,0,0,0,21.0,1
1738608,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,5,NONE,ALMA ST / COLE ST,-122.449433,37.763199,2008-02-22 18:00:00,1,18,2,8,-1.836970e-16,-0.222521,5.000000e-01,0,0,0,0,0,1,0,0,0,0,NaN,1
1246293,DRUG/NARCOTIC,POSSESSION OF BASE/ROCK COCAINE,1,"ARREST, BOOKED",400 Block of JONES ST,-122.412857,37.785580,2006-10-16 23:05:00,0,23,10,6,9.659258e-01,0.623490,5.000000e-01,0,0,0,0,0,0,0,0,0,1,21.0,0
2062811,OTHER OFFENSES,CONSPIRACY,3,JUVENILE BOOKED,1000 Block of HAYES ST,-122.433661,37.775465,2017-11-15 13:15:00,1,13,11,17,-9.659258e-01,-0.900969,8.660254e-01,0,0,0,0,0,1,0,0,0,0,NaN,0
1293057,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,1,NONE,1000 Block of PALOU AV,-122.380738,37.728276,2012-01-16 22:00:00,0,22,1,12,8.660254e-01,0.623490,8.660254e-01,1,0,0,0,0,0,0,0,0,0,1.0,0


### **Feature Selection**

In [23]:
train_df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'Resolution', 'Address', 'X', 'Y',
       'DateTime', 'IsDay', 'Hour', 'Month', 'Year', 'HourCos', 'DayOfWeekCos',
       'MonthCos', 'PdDistrict_BAYVIEW', 'PdDistrict_CENTRAL',
       'PdDistrict_INGLESIDE', 'PdDistrict_MISSION', 'PdDistrict_NORTHERN',
       'PdDistrict_PARK', 'PdDistrict_RICHMOND', 'PdDistrict_SOUTHERN',
       'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN', 'CategoryInt',
       'InIntersection'],
      dtype='object')

In [0]:
feature_cols = ['X', 'Y', 'IsDay', 'DayOfWeek', 'Month', 'Hour', 'Year', 'InIntersection',
                'PdDistrict_BAYVIEW', 'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE',
                'PdDistrict_MISSION', 'PdDistrict_NORTHERN', 'PdDistrict_PARK',
                'PdDistrict_RICHMOND', 'PdDistrict_SOUTHERN', 'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN']
target_col = 'CategoryInt'

train_x = train_df[feature_cols]
train_y = train_df[target_col]

test_ids = test_df['Id']
test_x = test_df[feature_cols]


In [25]:
train_x.sample(1)

,X,Y,IsDay,DayOfWeek,Month,Hour,Year,InIntersection,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
1220347,-122.432915,37.784353,1,5,4,19,8,1,0,0,0,0,1,0,0,0,0,0


In [26]:
test_x.sample(1)

,X,Y,IsDay,DayOfWeek,Month,Hour,Year,InIntersection,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
1814716,-122.427966,37.711582,1,6,1,13,15,0,0,0,1,0,0,0,0,0,0,0


#### **XGBOOST Training (Cross-Validation)**

In [27]:
type(train_x), type(train_y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from operator import itemgetter
import xgboost as xgb
import random
import time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
from numpy import genfromtxt
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
import datetime as dt

Import XGBoost and create the DMatrices

In [29]:
import xgboost as xgb
train_xgb = xgb.DMatrix(train_x, label=train_y)
test_xgb  = xgb.DMatrix(test_x)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Perform Cross-Validation

In [0]:
watchlist = [(train_xgb, 'train'), (test_xgb, 'eval')]

In [0]:
params = {
    'max_depth': 4,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 39,
}
num_boost_round = 500
early_stopping_rounds = 20

In [32]:
train_y_cleaned = train_y.dropna()
train_y_cleaned

1303833    37.0
38943       1.0
1236188     1.0
993800     21.0
1086513    38.0
           ... 
133235     16.0
371540      1.0
534847     21.0
1398900    16.0
1389548    20.0
Name: CategoryInt, Length: 993864, dtype: float64

In [0]:
CROSS_VAL = False
if CROSS_VAL:
    print('Doing Cross-validation ...')
    cv = xgb.cv(params, train_xgb, nfold=3, early_stopping_rounds=10, metrics='mlogloss', verbose_eval=True)
    cv


Fit the model and make the predictions

In [34]:
SUBMIT = not CROSS_VAL
if SUBMIT:
    print('Fitting Model ...')
    # Model Training
    m = xgb.train(params, train_xgb, 10)
    
    # Model Validation
    res = m.predict(test_xgb)

    # Precision-Recall Curve
    # score = average_precision_score(train_y_cleaned, res)
    # print('Area under the precision-recall curve: {:.6f}'.format(score))

    # Precision Score
    # res_round = res.round()
    # pre_score = precision_score(train_y_cleaned, res_round)
    # print('Precision score: {:.6f}'.format(pre_score))

    # Recall Score
    # rec_score = recall_score(train_y_cleaned, res_round)
    # print('Recall score: {:.6f}'.format(rec_score))

    cols = ['Id'] + cat_le.classes_
    submission = pd.DataFrame(res, columns=cat_le.classes_)
    submission.insert(0, 'Id', test_ids)
    submission.to_csv('submission.csv', index=False)
    print('Done Outputing !')
    submission.head()
else:
    print('NOT SUBMITING')

Fitting Model ...


ValueError: ignored

In [0]:
# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(train_y_cleaned.values, res)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
          lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [0]:
submission['Id']=submission.index

In [0]:
submission.head()

In [0]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(testy))]
# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)
# predict probabilities
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(testy, ns_probs)
lr_auc = roc_auc_score(testy, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()